In [1]:
import os
import sys
import datetime
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
# import argparse

2024-01-25 13:07:59.424593: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-25 13:07:59.444111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 13:07:59.444130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 13:07:59.444656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-25 13:07:59.448083: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def calculate_arena_size(tflite_path):
    '''Displays a model summary'''
    with tf.io.gfile.GFile(tflite_path, 'rb') as f:
      model_content = f.read()
    interpreter = tf.lite.Interpreter(model_content = model_content)
    interpreter.allocate_tensors()
    tensor_details = interpreter.get_tensor_details()
    op_details = interpreter._get_ops_details()

    get_tensor = lambda index : [t for t in tensor_details if t['index'] == index][0]
    get_node_tensors = lambda n : [get_tensor(t) for t in np.concatenate((n['inputs'],n['outputs']), axis= None)]    
    get_tensor_size = lambda t : np.prod(t['shape']) * np.dtype(t['dtype']).itemsize
    get_node_tensor_sizes = lambda o : np.sum([get_tensor_size(t) for t in get_node_tensors(o)])   
    get_max_node_size = lambda : np.max([get_node_tensor_sizes(o) for o in op_details])

    return get_max_node_size() 

    # return (interpreter.get_tensor_details(), \
    #   interpreter._get_ops_details())

In [4]:
modelPath = "../th/quantized_models/quant_retrained_model_95.4_20240125130552.tflite";
print(f"arena size is: {calculate_arena_size(modelPath)}")

arena size is: 318169
